<a href="https://colab.research.google.com/github/sharonmustri/balearic_power/blob/main/oxford_power_full_sharon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Create network model for Oxfordshire

In [2]:
%%capture
# install pandapower in colab
!pip install pandapower

In [3]:
# imports
import pandas as pd
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly

In [4]:
### to do

# check trafo specs -- gyubin checked
# check line specs -- gyubin checked 
# check pv gen -- multiply by capacity factor of 20%

In [5]:
# key variables
# loads
ADMD_MW = 2e-3
Q_MVAR = 24.2e-3/25
# solar
pv_MWp = 3.4e-3
pv_cf = 0.2

In [6]:
# create network
net = pp.create_empty_network() 

In [7]:
# layout characteristics
start_x = 0
start_y = 0

# distance between slack bus and 132/11 trafo
dist_slack_trafo11 = 0
# distance between 132/11 trafo and first house
dist_trafo11_hh = 1
# length of one side of household square plot
hh_side_km = 0.05
# distance between main road and first house (main road is 25m across)
dist_main_hh = 0.0125
# length of a full group of 25 households (3 rows of 8 plus 1, with a 10m road)
len_hh_group = 0.160
# space between each group of 25 households
len_bw_group = 0.02

In [8]:
# create bus dict
b = {}

In [9]:
# create bus at 132kV for grid connection
x, y = start_x, start_y
b[0] = pp.create_bus(net, vn_kv = 132, name = f"Bus 0", geodata = (x,y))

# create one grid connection point at b0
pp.create_ext_grid(net, bus=b[0], vm_pu=1, name = "Slack bus")

0

In [10]:
# create bus at 11kV for first step down
x, y = x + dist_slack_trafo11, y
b[1] = pp.create_bus(net, vn_kv = 11, name = f"Bus 1", geodata = (x,y))

# create one trafo b0 to b1 - from 132 to 11
hv_bus = b[0]
lv_bus = b[1]

sn_mva = 0.5
vn_hv_kv = 132
vn_lv_kv = 11
vk_percent = 10
vkr_percent = 1
pfe_kw = 5
i0_percent = 5

pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

0

In [11]:
# create buses at 11kV and 415V (4 each) for second step-down transformers
for i in range(2,6):
  if i == 2:
    x, y = x + dist_trafo11_hh + len_hh_group/2, y
  else: 
    x, y = x + len_hh_group + len_bw_group, y
  b[i] = pp.create_bus(net, vn_kv = 11, name = f"Bus {i}", geodata = (x,y))
  b[i*10] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i*10}", geodata = (x,y))

  # add bus for each of the 9 bundles up and down of the trafo grouping 25 houses
  for j in range(1,10):
    x, y = x, start_y + dist_main_hh + hh_side_km*j
    b[i*10+j] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i*10+j}", geodata = (x,y))
    b[i*10+j+0.1] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i*10+j+0.1}", geodata = (x,-y))
    
  # reset y for next trafo
  y = 0

In [12]:
# create four trafos - from 11kV to .415kV for b2/20, b3/30, b4/40, b5/50
sn_mva = 0.12
vn_hv_kv = 11
vn_lv_kv = 0.415
vk_percent = 10
vkr_percent = 1
pfe_kw = 1.2
i0_percent = 5

for i in range(2,6):
  hv_bus = b[i]
  lv_bus = b[i*10]
  pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

In [13]:
# create line at 11kV from b1 to b2 to b3 to b4 to b5
r_ohm_per_km = 0.411
x_ohm_per_km = 0.131
c_nf_per_km = 0.00031
max_i_ka = 0.1

for i in range(1,5):
  from_bus = b[i]
  to_bus = b[i+1]
  if i == 1:
    length_km = dist_trafo11_hh + len_hh_group/2
  else:
    length_km = len_hh_group + len_bw_group
  pp.create_line_from_parameters(net, from_bus=from_bus, to_bus=to_bus, length_km=length_km, r_ohm_per_km=r_ohm_per_km, x_ohm_per_km=x_ohm_per_km, c_nf_per_km=c_nf_per_km, max_i_ka=max_i_ka) 

In [14]:
# create upward/downward lines at 415V from each trafo
r_ohm_per_km = 0.5     
x_ohm_per_km = 0.09
c_nf_per_km = 1e-9
max_i_ka = 0.15

for i in range(2,6):
  for j in range(1,10):
    if j == 1:
      length_km = hh_side_km + dist_main_hh
    else: 
      length_km = hh_side_km
    # upward
    from_bus = b[i*10+j-1]  
    to_bus = b[i*10+j]
    pp.create_line_from_parameters(net, from_bus=from_bus, to_bus=to_bus, length_km=length_km, r_ohm_per_km=r_ohm_per_km, x_ohm_per_km=x_ohm_per_km, c_nf_per_km=c_nf_per_km, max_i_ka=max_i_ka)     
    # downward
    if j == 1:
      from_bus = b[i*10]
    else:
      from_bus = b[i*10+j-1+0.1]  
    to_bus = b[i*10+j+0.1]
    pp.create_line_from_parameters(net, from_bus=from_bus, to_bus=to_bus, length_km=length_km, r_ohm_per_km=r_ohm_per_km, x_ohm_per_km=x_ohm_per_km, c_nf_per_km=c_nf_per_km, max_i_ka=max_i_ka) 

In [15]:
simple_plotly(net)

In [16]:
# create loads
for i in range(2,6):
  for j in range(1,10):
    if j == 9:
      p_mw = ADMD_MW
      q_mvar = Q_MVAR
    else: 
      p_mw = ADMD_MW * 3
      q_mvar = Q_MVAR * 3
    # upward
    pp.create_load(net, b[i*10+j], p_mw=p_mw, q_mvar=q_mvar)
    # downward
    pp.create_load(net, b[i*10+j+0.1], p_mw=p_mw, q_mvar=q_mvar)

In [17]:
# create pv gen
for i in range(2,6):
  for j in range(1,10):
    if j == 9:
      p_mw = (-1) * pv_MWp * pv_cf
      q_mvar = 0
    else: 
      p_mw = (-1) * pv_MWp * pv_cf * 3
      q_mvar = 0 * 3
    # upward
    pp.create_load(net, b[i*10+j], p_mw=p_mw, q_mvar=q_mvar)
    # downward
    pp.create_load(net, b[i*10+j+0.1], p_mw=p_mw, q_mvar=q_mvar)

In [19]:
# run network
pp.runpp(net, algorithm='nr');

In [20]:
# check is any bus has vm_pu below 0.8
sum(net.res_bus.vm_pu<0.8)

0

In [21]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.289265,-0.302614
1,0.937813,-3.137615,0.000000,0.000000
2,0.936406,-3.100087,0.000000,0.000000
3,0.879735,-6.780604,0.000000,0.000000
4,0.871786,-6.533081,0.003960,0.002904
...,...,...,...,...
77,0.849206,-5.812366,0.003960,0.002904
78,0.848183,-5.778630,0.003960,0.002904
79,0.848183,-5.778630,0.003960,0.002904
80,0.847927,-5.770184,0.001320,0.000968
